In [1]:
import numpy as np
from time import time
import healpy as hp
from multiprocessing import Pool
import tables
import sqlite3
import sys
import os
#from lsd import DB
#from lsd import bounds as lsdbounds
# various functions are defined here
import pmfuns

import calcMedianAndResiduals as ca
import pandas as pd
import matplotlib.pyplot as plt
import spherical_to_tangential as s2t
import random
import esutil
from esutil.numpy_util import match
from astropy.io import fits
from astropy.table import Table
import matplotlib
from esutil.coords import sphdist

In [2]:
nside = 2**24# pixelizes the sky
rootdir = "./data/" 

In [3]:
input_file = './data_per/hlsp_phat_hst_acs-wfc_12057-m31-b09-f01_f475w-f814w_v1_gst.fits'
hdu_list = fits.open(input_file, memmap=True)
evt_data = Table(hdu_list[1].data)
CRA = np.median(evt_data['RA'])
CDE = np.median(evt_data['DEC'])

In [4]:
table_gal = pd.read_csv('./data/B9ComB11_gal3.0.csv')
table_star = pd.read_csv('./data/B9ComB11_star3.0.csv')

chunkNo = 0
refDB = rootdir + "referenceGalDB%d"%chunkNo
correctedStarsName = "correctedStarMock" + "%d" %chunkNo
h5correctedStarsFile = rootdir + "%s.h5" %correctedStarsName

maskForGal  = (table_gal['objid']!=0)
maskForStar = (table_star['objid']!=0)

In [5]:
galaxyIDs = table_gal['obj_id'][maskForGal]
raGalaxy   = table_gal['ra'][maskForGal]
decGalaxy  = table_gal['dec'][maskForGal]
mjdsGalaxy = table_gal['mjd'][maskForGal]


starIDs  = table_star['obj_id'][maskForStar]
raStar   = table_star['ra'][maskForStar]
decStar  = table_star['dec'][maskForStar]
#raErrStar   = table['raErr'][maskForStar]#/(3600*1000)
#decErrStar  = table['decErr'][maskForStar]#/(3600*1000)
mjdsStar = table_star['mjd'][maskForStar]
rMagStar = table_star['mr'][maskForStar]
nObsStar = np.zeros(sum(maskForStar)) + 2 #table.col('nObs')[maskForStar]

In [6]:
#print 'detection number for all the objs', len(table['obj_id'])
print 'detection number for all the gals', len(galaxyIDs)
print 'detection number for all the stars', len(starIDs)
print 'raStar, decStar', raStar[0:2], decStar[0:2]

detection number for all the gals 2542
detection number for all the stars 645140
raStar, decStar 0    11.209234
1    11.209236
Name: ra, dtype: float64 0    41.655584
1    41.655580
Name: dec, dtype: float64


In [7]:
mjdSortedStar  = np.sort(mjdsStar)
deltaTStar     = mjdSortedStar[0:-1] - mjdSortedStar[1:]
tBreakAtStar   = np.where(deltaTStar < (-100.0))[0]
mjdBreakAtStar = mjdSortedStar[tBreakAtStar + 1]

In [8]:
objIDarray, finalRaArray, finalDecArray, medianRaError, medianDecError = pmfuns.readDB(refDB)
print 'finalRaArray, finalDecArray', finalRaArray[0:2], finalDecArray[0:2]

finalRaArray, finalDecArray [11.1890045 11.2079145] [41.6011175 41.6010905]


In [9]:
phiForObj   = (finalRaArray*np.pi)/180 
thetaForObj = (90 - finalDecArray)* (np.pi/180) 
pixelIndexForObj = hp.ang2pix(nside, thetaForObj, phiForObj) 	    
pixelIndexArray = np.unique(pixelIndexForObj)
theta, phi = hp.pix2ang(nside, pixelIndexArray)
pixelRa  = 180*phi/np.pi
pixelDec = 90 - theta*180/np.pi
print "Going to process %d pixels." % pixelIndexArray.size
phiForStar   = (raStar*np.pi)/180 
thetaForStar = (90 - decStar)* (np.pi/180) 
pixelIndexForStar = hp.ang2pix(nside, thetaForStar, phiForStar) 
print "mjdBreakAtStar:", mjdBreakAtStar

pixelIndexForStar_l = []
for i in pixelIndexForStar:
    pixelIndexForStar_l.append(int(i*10**(-9)))
print len(pixelIndexForStar_l)

gnomonic = True
packParameterList = [ (pickPixelNo, pixelRa[index], pixelDec[index],objIDarray, finalRaArray, finalDecArray, \
np.array(galaxyIDs), np.array(raGalaxy), np.array(decGalaxy),mjdsGalaxy , starIDs, mjdsStar, raStar, decStar, \
rMagStar, nObsStar, pixelIndexForStar, pixelIndexForStar_l, mjdBreakAtStar, gnomonic, CRA, CDE) \
for index, pickPixelNo in enumerate(pixelIndexArray)]

Going to process 1271 pixels.
mjdBreakAtStar: [56508]
645140


In [10]:
pmfuns.correctedStars(packParameterList[0:10000], h5correctedStarsFile)

raGalInRadius, decGalInRadius: [11.215333 11.215332 11.215183 ... 11.185263 11.222282 11.222285] [41.659115 41.659116 41.658939 ... 41.630005 41.622697 41.622693]
galIDinRadius: [ 370  370  376 ... 1259  627  627]
galCounter after loop 599
raGalInRadius, decGalInRadius: [11.221554 11.22156  11.222209 ... 11.196169 11.1957   11.195707] [41.629223 41.629217 41.628707 ... 41.640117 41.618938 41.61894 ]
galIDinRadius: [ 573  573  566 ...  981 1174 1174]
galCounter after loop 599
raGalInRadius, decGalInRadius: [11.217279 11.217279 11.217088 ... 11.190236 11.195974 11.195958] [41.610771 41.610763 41.611166 ... 41.616624 41.624585 41.624592]
galIDinRadius: [ 856  856  864 ...   18 1103 1103]
galCounter after loop 599
star in Pixel: 637
star in Pixel: 22
sum number of resIndexInPixel 637
star in Pixel: 537
raGalInRadius, decGalInRadius: [11.206907 11.206912 11.20733  ... 11.213877 11.230652 11.230656] [41.629184 41.629179 41.62961  ... 41.648421 41.620102 41.620098]
galIDinRadius: [874 874 850

galCounter after loop 599
star in Pixel: 92
sum number of resIndexInPixel 92
raGalInRadius, decGalInRadius: [11.210156 11.210138 11.210791 ... 11.219775 11.226331 11.226332] [41.657528 41.657519 41.657161 ... 41.623967 41.625355 41.625352]
galIDinRadius: [521 521 501 ... 660 519 519]
galCounter after loop 599
star in Pixel: 661
sum number of resIndexInPixel 661
raGalInRadius, decGalInRadius: [11.204911 11.204906 11.205061 ... 11.22461  11.214264 11.214268] [41.62871  41.628708 41.627631 ... 41.6421   41.647367 41.647362]
galIDinRadius: [918 918 925 ... 359 561 561]
galCounter after loop 599
star in Pixel: 552
sum number of resIndexInPixel 552
raGalInRadius, decGalInRadius: [11.177959 11.177963 11.177555 ... 11.208182 11.184944 11.184949] [41.610508 41.610505 41.610682 ... 41.629242 41.63939  41.639389]
galIDinRadius: [ 266  266  219 ...  834 1181 1181]
galCounter after loop 599
star in Pixel: 690
star in Pixel: 75
sum number of resIndexInPixel 690
raGalInRadius, decGalInRadius: [11.222

sum number of resIndexInPixel 88
sum number of resIndexInPixel 512
raGalInRadius, decGalInRadius: [11.216788 11.216795 11.215811 ... 11.21639  11.205439 11.205443] [41.656243 41.656242 41.655949 ... 41.621964 41.622912 41.622911]
galIDinRadius: [374 374 404 ... 744 956 956]
raGalInRadius, decGalInRadius: [11.183851 11.183867 11.184592 ... 11.189027 11.209159 11.209167] [41.609938 41.609937 41.610703 ... 41.636131 41.628469 41.628466]
galIDinRadius: [ 163  163  149 ... 1130  852  852]
galCounter after loop 599
galCounter after loop 599
sum number of resIndexInPixel 666
raGalInRadius, decGalInRadius: [11.218727 11.218721 11.218696 ... 11.194279 11.194993 11.195003] [41.628327 41.628322 41.628539 ... 41.638616 41.617119 41.617118]
galIDinRadius: [ 637  637  619 ... 1023 1210 1210]
galCounter after loop 599
star in Pixel: 665
sum number of resIndexInPixel 665
raGalInRadius, decGalInRadius: [11.195727 11.195758 11.195438 ... 11.221454 11.197456 11.19744 ] [41.6283   41.628302 41.6275   ... 

galIDinRadius: [ 348  348  353 ...  442 1128 1128]
galCounter after loop 599
star in Pixel: 649
star in Pixel: 556
sum number of resIndexInPixel 649
raGalInRadius, decGalInRadius: [11.185951 11.185966 11.186657 ... 11.216493 11.201568 11.201576] [41.627971 41.627972 41.627646 ... 41.625684 41.608214 41.608209]
galIDinRadius: [1257 1257 1256 ...  688 1170 1170]
sum number of resIndexInPixel 556
galCounter after loop 599
raGalInRadius, decGalInRadius: [11.185673 11.185677 11.186182 ... 11.214636 11.208178 11.208182] [41.609587 41.609588 41.609719 ... 41.623728 41.629247 41.629242]
galIDinRadius: [150 150 134 ... 771 834 834]
galCounter after loop 599
star in Pixel: 115
sum number of resIndexInPixel 115
raGalInRadius, decGalInRadius: [11.20973  11.209722 11.210427 ... 11.17768  11.209636 11.209682] [41.655075 41.65508  41.654802 ... 41.633704 41.62295  41.622915]
galIDinRadius: [544 544 529 ...  71 893 893]
galCounter after loop 599
star in Pixel: 556
sum number of resIndexInPixel 556
raG

star in Pixel: 518
star in Pixel: 659
sum number of resIndexInPixel 518
raGalInRadius, decGalInRadius: [11.194929 11.194934 11.19382  ... 11.22274  11.221647 11.22164 ] [41.60913  41.609121 41.60921  ... 41.618333 41.6201   41.620095]
galIDinRadius: [1271 1271   21 ...  659  648  648]
galCounter after loop 599
sum number of resIndexInPixel 659
raGalInRadius, decGalInRadius: [11.214732 11.214716 11.21459  ... 11.23933  11.195174 11.195157] [41.627571 41.627566 41.62853  ... 41.618733 41.641847 41.641877]
galIDinRadius: [707 707 699 ... 318 971 971]
galCounter after loop 599
star in Pixel: 180
sum number of resIndexInPixel 180
raGalInRadius, decGalInRadius: [11.20392  11.203922 11.202205 ... 11.178416 11.221715 11.221713] [41.654358 41.654358 41.65471  ... 41.629934 41.626337 41.626329]
galIDinRadius: [656 656 675 ... 100 597 597]
galCounter after loop 599
star in Pixel: 493
star in Pixel: 652
sum number of resIndexInPixel 493
raGalInRadius, decGalInRadius: [11.220081 11.220077 11.219509

...

galIDinRadius: [114 114 112 ... 699 640 640]
galCounter after loop 599
star in Pixel: 632
sum number of resIndexInPixel 632
raGalInRadius, decGalInRadius: [11.192396 11.192396 11.19324  ... 11.211436 11.186771 11.186779] [41.613678 41.613672 41.614853 ... 41.629528 41.634635 41.634627]
galIDinRadius: [   1    1 1258 ...  773 1190 1190]
galCounter after loop 599
star in Pixel: 656
sum number of resIndexInPixel 656
raGalInRadius, decGalInRadius: [11.220868 11.220856 11.221549 ... 11.198548 11.201408 11.201423] [41.632463 41.632461 41.632241 ... 41.61921  41.616877 41.616881]
galIDinRadius: [ 565  565  536 ... 1093 1073 1073]
galCounter after loop 599
star in Pixel: 616
sum number of resIndexInPixel 616
raGalInRadius, decGalInRadius: [11.240496 11.240497 11.238387 ... 11.19576  11.196608 11.196618] [41.61367  41.613664 41.613579 ... 41.611834 41.620598 41.620598]
galIDinRadius: [ 340  340  427 ... 1240 1134 1134]
galCounter after loop 599
star in Pixel: 656
sum number of resIndexInPixel 6

galCounter after loop 599
star in Pixel: 576
sum number of resIndexInPixel 576
raGalInRadius, decGalInRadius: [11.221214 11.221212 11.220583 ... 11.192451 11.192109 11.192114] [41.613044 41.613039 41.613238 ... 41.606693 41.60738  41.607378]
galIDinRadius: [726 726 746 ...  58  54  54]
galCounter after loop 599
star in Pixel: 643
sum number of resIndexInPixel 643
raGalInRadius, decGalInRadius: [11.167999 11.168011 11.169895 ... 11.212475 11.202947 11.202948] [41.631563 41.631561 41.632209 ... 41.627212 41.610542 41.610541]
galIDinRadius: [ 261  261  183 ...  751 1101 1101]
galCounter after loop 599
star in Pixel: 576
sum number of resIndexInPixel 576
raGalInRadius, decGalInRadius: [11.235433 11.235457 11.23516  ... 11.196651 11.200924 11.200926] [41.613052 41.613028 41.613248 ... 41.621971 41.629078 41.629073]
galIDinRadius: [ 462  462  466 ... 1109  993  993]
galCounter after loop 599
star in Pixel: 709
sum number of resIndexInPixel 709
raGalInRadius, decGalInRadius: [11.220215 11.220

raGalInRadius, decGalInRadius: [11.224178 11.224176 11.224174 ... 11.214327 11.233033 11.233035] [41.61255  41.612544 41.611642 ... 41.635653 41.635907 41.635902]
galIDinRadius: [685 685 679 ... 644 291 291]
galCounter after loop 599
star in Pixel: 568
sum number of resIndexInPixel 568
raGalInRadius, decGalInRadius: [11.214998 11.21503  11.214229 ... 11.222804 11.225527 11.225515] [41.630501 41.630461 41.629946 ... 41.610991 41.611688 41.611697]
galIDinRadius: [676 676 693 ... 727 658 658]
galCounter after loop 599
star in Pixel: 612
sum number of resIndexInPixel 612
raGalInRadius, decGalInRadius: [11.180502 11.180512 11.18076  ... 11.213641 11.184944 11.184949] [41.612464 41.612463 41.61363  ... 41.62351  41.63939  41.639389]
galIDinRadius: [ 179  179  172 ...  826 1181 1181]
galCounter after loop 599
star in Pixel: 650
sum number of resIndexInPixel 650
raGalInRadius, decGalInRadius: [11.233847 11.233845 11.235485 ... 11.200715 11.197596 11.197566] [41.630465 41.630463 41.632617 ... 4

galIDinRadius: [1240 1240 1237 ...  667  707  707]
galCounter after loop 599
sum number of resIndexInPixel 679
raGalInRadius, decGalInRadius: [11.17839  11.178416 11.178488 ... 11.192409 11.200584 11.200596] [41.62994  41.629934 41.630227 ... 41.605023 41.651314 41.651263]
galIDinRadius: [100 100  92 ...  69 753 753]
galCounter after loop 599
star in Pixel: 568
star in Pixel: 679
sum number of resIndexInPixel 568
raGalInRadius, decGalInRadius: [11.219164 11.219176 11.218898 ... 11.190482 11.191335 11.19132 ] [41.611786 41.611781 41.611724 ... 41.615556 41.605103 41.605109]
galIDinRadius: [791 791 804 ...  10  98  98]
galCounter after loop 599
sum number of resIndexInPixel 679
raGalInRadius, decGalInRadius: [11.185026 11.185028 11.185255 ... 11.200146 11.216556 11.216558] [41.629931 41.62993  41.630005 ... 41.650784 41.632757 41.632753]
galIDinRadius: [1262 1262 1259 ...  762  629  629]
galCounter after loop 599
star in Pixel: 605
star in Pixel: 679
sum number of resIndexInPixel 605
raG

raGalInRadius, decGalInRadius: [11.160847 11.160874 11.160591 ... 11.207655 11.206726 11.206726] [41.629406 41.629385 41.629294 ... 41.641379 41.643294 41.643293]
galIDinRadius: [248 248 189 ... 705 700 700]
galCounter after loop 599
star in Pixel: 570
star in Pixel: 644
sum number of resIndexInPixel 570
raGalInRadius, decGalInRadius: [11.223213 11.223209 11.222822 ... 11.193849 11.191249 11.191254] [41.611429 41.611427 41.610978 ... 41.60192  41.610114 41.610109]
galIDinRadius: [710 710 727 ...  75  42  42]
galCounter after loop 599
sum number of resIndexInPixel 644
raGalInRadius, decGalInRadius: [11.180567 11.180568 11.179826 ... 11.214735 11.204424 11.204426] [41.629308 41.62931  41.629588 ... 41.626486 41.610762 41.610754]
galIDinRadius: [  81   81   68 ...  716 1070 1070]
galCounter after loop 599
star in Pixel: 570
sum number of resIndexInPixel 570
raGalInRadius, decGalInRadius: [11.200734 11.200707 11.200468 ... 11.215821 11.192768 11.19277 ] [41.611412 41.611404 41.611769 ... 4

In [10]:
for i in range(len(pixelIndexArray)):
    for j in range(len(pixelIndexForStar)):
        if pixelIndexArray[i] == pixelIndexForStar[j]:
            print(pixelIndexArray[i], pixelIndexForStar[j],i,j)

(566298184508801, 566298184508801, 3, 103938)
(566552929753793, 566552929753793, 199, 246956)
(566591316022929, 566591316022929, 231, 16697)
(566633594607582, 566633594607582, 264, 53779)
(566950818210371, 566950818210371, 516, 610505)
(567060541205250, 567060541205250, 667, 158799)
(567199121008924, 567199121008924, 859, 9843)
(567336761283979, 567336761283979, 998, 25708)
(567415479981528, 567415479981528, 1078, 496852)
(567443330159814, 567443330159814, 1105, 350721)
(567703913878156, 567703913878156, 1253, 135769)


In [11]:
pixelIndexForStar_l = []
for i in pixelIndexForStar:
    pixelIndexForStar_l.append(int(i*10**(-9)))
print len(pixelIndexForStar_l)

645140


In [18]:
for pickPixelNo in enumerate(pixelIndexArray):
    #print(type(pickPixelNo[1]))
   # resIndexInPixel =  int(pickPixelNo[1]*10**(-14)) == pixelIndexForStar_lf
    resIndexInPixel_all = []
    for j in pixelIndexForStar_l:
        resIndexInPixel =  int(pickPixelNo[1]*10**(-9)) == j
        resIndexInPixel_all.append(resIndexInPixel)
    #print pickPixelNo, pixelIndexForStar
        #if resIndexInPixel != False:
    print "star in Pixel:", np.sum(resIndexInPixel_all)
        #resIndexInPixel_all.append(resIndexInPixel)

star in Pixel: 22
star in Pixel: 21
star in Pixel: 37
star in Pixel: 39
star in Pixel: 69
star in Pixel: 64
star in Pixel: 64
star in Pixel: 92
star in Pixel: 92
star in Pixel: 75
star in Pixel: 63
star in Pixel: 83
star in Pixel: 83
star in Pixel: 73
star in Pixel: 82
star in Pixel: 82
star in Pixel: 82
star in Pixel: 88
star in Pixel: 88
star in Pixel: 109
star in Pixel: 111
star in Pixel: 99
star in Pixel: 105
star in Pixel: 105
star in Pixel: 105
star in Pixel: 116
star in Pixel: 170
star in Pixel: 121
star in Pixel: 115
star in Pixel: 124
star in Pixel: 140
star in Pixel: 170
star in Pixel: 147
star in Pixel: 152
star in Pixel: 139
star in Pixel: 175
star in Pixel: 171
star in Pixel: 180
star in Pixel: 201
star in Pixel: 201
star in Pixel: 149
star in Pixel: 167
star in Pixel: 168
star in Pixel: 168
star in Pixel: 197
star in Pixel: 171
star in Pixel: 171
star in Pixel: 182
star in Pixel: 222
star in Pixel: 197
star in Pixel: 197
star in Pixel: 228
star in Pixel: 228
star in Pixel

star in Pixel: 583
star in Pixel: 634
star in Pixel: 634
star in Pixel: 634
star in Pixel: 638
star in Pixel: 656
star in Pixel: 656
star in Pixel: 641
star in Pixel: 694
star in Pixel: 620
star in Pixel: 620
star in Pixel: 596
star in Pixel: 574
star in Pixel: 632
star in Pixel: 632
star in Pixel: 652
star in Pixel: 640
star in Pixel: 629
star in Pixel: 643
star in Pixel: 643
star in Pixel: 709
star in Pixel: 709
star in Pixel: 709
star in Pixel: 662
star in Pixel: 662
star in Pixel: 640
star in Pixel: 574
star in Pixel: 653
star in Pixel: 653
star in Pixel: 697
star in Pixel: 619
star in Pixel: 678
star in Pixel: 711
star in Pixel: 568
star in Pixel: 650
star in Pixel: 650
star in Pixel: 647
star in Pixel: 646
star in Pixel: 645
star in Pixel: 732
star in Pixel: 732
star in Pixel: 740
star in Pixel: 621
star in Pixel: 621
star in Pixel: 621
star in Pixel: 654
star in Pixel: 617
star in Pixel: 679
star in Pixel: 679
star in Pixel: 679
star in Pixel: 635
star in Pixel: 635
star in Pixe

star in Pixel: 706
star in Pixel: 632
star in Pixel: 632
star in Pixel: 642
star in Pixel: 601
star in Pixel: 621
star in Pixel: 563
star in Pixel: 563
star in Pixel: 563
star in Pixel: 654
star in Pixel: 654
star in Pixel: 656
star in Pixel: 656
star in Pixel: 656
star in Pixel: 647
star in Pixel: 647
star in Pixel: 643
star in Pixel: 643
star in Pixel: 643
star in Pixel: 609
star in Pixel: 609
star in Pixel: 609
star in Pixel: 633
star in Pixel: 699
star in Pixel: 661
star in Pixel: 661
star in Pixel: 635
star in Pixel: 635
star in Pixel: 672
star in Pixel: 608
star in Pixel: 608
star in Pixel: 608
star in Pixel: 643
star in Pixel: 643
star in Pixel: 666
star in Pixel: 599
star in Pixel: 599
star in Pixel: 624
star in Pixel: 624
star in Pixel: 599
star in Pixel: 583
star in Pixel: 647
star in Pixel: 651
star in Pixel: 652
star in Pixel: 583
star in Pixel: 583
star in Pixel: 633
star in Pixel: 616
star in Pixel: 590
star in Pixel: 625
star in Pixel: 592
star in Pixel: 591
star in Pixe

In [11]:
pmfuns.correctedStars(packParameterList, h5correctedStarsFile)

raGalInRadius, decGalInRadius: [11.215333 11.215332 11.215183 ... 11.185263 11.222282 11.222285] [41.659115 41.659116 41.658939 ... 41.630005 41.622697 41.622693]
galIDinRadius: [ 370  370  376 ... 1259  627  627]
galCounter after loop 599
pixelNo: 566275703039681 0         566353482211921
1         566353616429649
2         566893574348394
3         566893641457258
4         567243547068777
5         567243614177642
6         566971353523998
7         566971353523998
8         567398031684449
9         567398098793313
10        567100403865426
11        567100470974291
12        567182612232770
13        567182612232771
14        567389777288857
15        567389844397721
16        567548556856837
17        567548623965701
18        567087384750402
19        567087518968130
20        566530045628548
21        566530112737412
22        566975245835425
23        566975312944291
24        566841833415295
25        566841900524160
26        567057252868341
27        567057387086069
28     

Name: dec, Length: 645140, dtype: int64

sum of star number in Pixel: 0
sum of star number in Pixel: 0
sum number of resIndexInPixel 0
sum number of resIndexInPixel 0
raGalInRadius, decGalInRadius: [11.217279 11.217279 11.217088 ... 11.190236 11.195974 11.195958] [41.610771 41.610763 41.611166 ... 41.616624 41.624585 41.624592]
galIDinRadius: [ 856  856  864 ...   18 1103 1103]
raGalInRadius, decGalInRadius: [11.213613 11.213611 11.21417  ... 11.200448 11.178052 11.178112] [41.658093 41.658095 41.658352 ... 41.624048 41.634588 41.634601]
galIDinRadius: [ 434  434  416 ... 1052   59   59]
galCounter after loop 599
pixelNo: 567340854929452 0         566353482211921
1         566353616429649
2         566893574348394
3         566893641457258
4         567243547068777
5         567243614177642
6         566971353523998
7         566971353523998
8         567398031684449
9         567398098793313
10        567100403865426
11        567100470974291
12        567182612232770
13        567182

Name: dec, Length: 645140, dtype: int64
sum number of resIndexInPixel 0
raGalInRadius, decGalInRadius: [11.215132 11.215136 11.215421 ... 11.179218 11.212893 11.21291 ] [41.657897 41.657913 41.657359 ... 41.634461 41.622316 41.622289]
galIDinRadius: [406 406 410 ...  35 821 821]
sum of star number in Pixel: 0
raGalInRadius, decGalInRadius: [11.220841 11.220848 11.221106 ... 11.190854 11.190684 11.190681] [41.610738 41.610732 41.611038 ... 41.616084 41.605888 41.605891]
galCounter after loop 599
galIDinRadius: [768 768 749 ...  20  96  96]
pixelNo: 566302412367516 0         566353482211921
1         566353616429649
2         566893574348394
3         566893641457258
4         567243547068777
5         567243614177642
6         566971353523998
7         566971353523998
8         567398031684449
9         567398098793313
10        567100403865426
11        567100470974291
12        567182612232770
13        567182612232771
14        567389777288857
15        567389844397721
16        5675

Name: dec, Length: 645140, dtype: int64
sum number of resIndexInPixel 0
sum of star number in Pixel: 0
sum number of resIndexInPixel 0
raGalInRadius, decGalInRadius: [11.211744 11.211683 11.210791 ... 11.223007 11.209636 11.209682] [41.657828 41.657823 41.657161 ... 41.623942 41.62295  41.622915]
raGalInRadius, decGalInRadius: [11.200924 11.200926 11.200356 ... 11.222167 11.188874 11.188876] [41.629078 41.629073 41.629452 ... 41.617332 41.611464 41.611462]
galIDinRadius: [464 464 501 ... 594 893 893]
galIDinRadius: [ 993  993 1016 ...  668   72   72]
sum number of resIndexInPixel 0
galCounter after loop 599
galCounter after loop 599
pixelNo: 566304090088479 0         566353482211921
1         566353616429649
2         566893574348394
3         566893641457258
4         567243547068777
5         567243614177642
6         566971353523998
7         566971353523998
8         567398031684449
9         567398098793313
10        567100403865426
11        567100470974291
12        567182612232

Name: dec, Length: 645140, dtype: int64
sum number of resIndexInPixel 0
sum of star number in Pixel: 0
raGalInRadius, decGalInRadius: [11.180552 11.180558 11.181351 ... 11.215335 11.213618 11.213641] [41.610621 41.61062  41.610023 ... 41.620689 41.623508 41.62351 ]
galIDinRadius: [256 256 236 ... 783 826 826]
sum number of resIndexInPixel 0
raGalInRadius, decGalInRadius: [11.215968 11.215969 11.215421 ... 11.221364 11.181961 11.181962] [41.657622 41.65762  41.657359 ... 41.622377 41.632838 41.632837]
galCounter after loop 599
galIDinRadius: [379 379 410 ... 670  13  13]
pixelNo: 567344076148078 0         566353482211921
1         566353616429649
2         566893574348394
3         566893641457258
4         567243547068777
5         567243614177642
6         566971353523998
7         566971353523998
8         567398031684449
9         567398098793313
10        567100403865426
11        567100470974291
12        567182612232770
13        567182612232771
14        567389777288857
15      

...

Name: dec, Length: 645140, dtype: int64
sum of star number in Pixel: 0
galCounter after loop 599
pixelNo: 567334949350133 0         566353482211921
1         566353616429649
2         566893574348394
3         566893641457258
4         567243547068777
5         567243614177642
6         566971353523998
7         566971353523998
8         567398031684449
9         567398098793313
10        567100403865426
11        567100470974291
12        567182612232770
13        567182612232771
14        567389777288857
15        567389844397721
16        567548556856837
17        567548623965701
18        567087384750402
19        567087518968130
20        566530045628548
21        566530112737412
22        566975245835425
23        566975312944291
24        566841833415295
25        566841900524160
26        567057252868341
27        567057387086069
28        567280792500809
29        567280792500809
               ...       
645110    566864381986769
645111    566864449095634
645112    5664848813

Name: dec, Length: 645140, dtype: int64
sum of star number in Pixel: 1
raGalInRadius, decGalInRadius: [11.161278 11.161284 11.160847 ... 11.193849 11.198222 11.198223] [41.629409 41.629419 41.629406 ... 41.60192  41.605212 41.605212]
galIDinRadius: [250 250 248 ...  75   2   2]
sum number of resIndexInPixel 1
galCounter after loop 599
pixelNo: 566929947344742 0         566353482211921
1         566353616429649
2         566893574348394
3         566893641457258
4         567243547068777
5         567243614177642
6         566971353523998
7         566971353523998
8         567398031684449
9         567398098793313
10        567100403865426
11        567100470974291
12        567182612232770
13        567182612232771
14        567389777288857
15        567389844397721
16        567548556856837
17        567548623965701
18        567087384750402
19        567087518968130
20        566530045628548
21        566530112737412
22        566975245835425
23        566975312944291
24        5668

Name: dec, Length: 645140, dtype: int64
sum of star number in Pixel: 0
sum number of resIndexInPixel 0
raGalInRadius, decGalInRadius: [11.210621 11.210623 11.210103 ... 11.187019 11.235212 11.235209] [41.629253 41.62925  41.628754 ... 41.638562 41.621377 41.62137 ]
galIDinRadius: [ 774  774  796 ... 1148  365  365]
galCounter after loop 599
pixelNo: 566933504123731 0         566353482211921
1         566353616429649
2         566893574348394
3         566893641457258
4         567243547068777
5         567243614177642
6         566971353523998
7         566971353523998
8         567398031684449
9         567398098793313
10        567100403865426
11        567100470974291
12        567182612232770
13        567182612232771
14        567389777288857
15        567389844397721
16        567548556856837
17        567548623965701
18        567087384750402
19        567087518968130
20        566530045628548
21        566530112737412
22        566975245835425
23        566975312944291
24      

Process PoolWorker-20:
Process PoolWorker-19:
Process PoolWorker-1:
Process PoolWorker-3:
Process PoolWorker-5:
Process PoolWorker-10:
Process PoolWorker-6:
Process PoolWorker-7:
Process PoolWorker-2:
Process PoolWorker-13:
Process PoolWorker-11:
Process PoolWorker-9:
Process PoolWorker-12:
Process PoolWorker-17:
Process PoolWorker-15:
Traceback (most recent call last):
Process PoolWorker-18:
Process PoolWorker-14:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process PoolWorker-4:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/njl/anaconda2/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
Traceback (most recent call last):
Traceback

  File "/home/njl/anaconda2/lib/python2.7/multiprocessing/queues.py", line 374, in get
  File "/home/njl/anaconda2/lib/python2.7/multiprocessing/queues.py", line 374, in get
  File "/home/njl/anaconda2/lib/python2.7/multiprocessing/queues.py", line 374, in get
    self._target(*self._args, **self._kwargs)
  File "/home/njl/anaconda2/lib/python2.7/multiprocessing/queues.py", line 374, in get
  File "/home/njl/anaconda2/lib/python2.7/multiprocessing/queues.py", line 374, in get
  File "/home/njl/anaconda2/lib/python2.7/multiprocessing/queues.py", line 374, in get
  File "/home/njl/anaconda2/lib/python2.7/multiprocessing/queues.py", line 374, in get
  File "/home/njl/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/home/njl/anaconda2/lib/python2.7/multiprocessing/queues.py", line 374, in get
    racquire()
    racquire()
    racquire()
    racquire()
    racquire()
    racquire()
    racquire()
    racquire()
    racquire()
    racquire()
    racquire()
    r

In [27]:
pixelIndexArray

array([566275703039681, 566279595353766, 566292547364329, ...,
       567771425396092, 567802161255283, 567805114045176])

In [31]:
a = int(pixelIndexForStar[1]*10**(-8))
a

5663536

In [14]:
len(pixelIndexForStar)

645140

In [14]:
NGAL = 600
for i, pickPixelNo in enumerate(pixelIndexArray):
    pixelNo = pickPixelNo
    pixRa = pixelRa[i]
    pixDec = pixelDec[i]
    galIDfinal = np.array(objIDarray)
    raFinalGal = finalRaArray
    decFinalGal = finalDecArray
    galIDs = np.array(galaxyIDs)
    raGal = np.array(raGalaxy)
    decGal = np.array(decGalaxy)
    galMjd = mjdsGalaxy
    starIDs = starIDs
    starMjds = mjdsStar
    raStar = raStar
    decStar = decStar
    rMagStar = rMagStar
    nObsStar = nObsStar
    pixAllStar = pixelIndexForStar
    mjdBreakAt = mjdBreakAtStar
    ra0 = CRA
    dec0 = CDE
    distTmp = sphdist(pixelRa, pixelDec, raFinalGal, decFinalGal)
    '''angSepMask = (distTmp) <= (60.0/60.0)
    if(sum(angSepMask)>600):
        angSepMask = np.argsort(distTmp)
        angSepMask = angSepMask[0:NGAL]
    elif(sum(angSepMask)<100):
        angSepMask = np.argsort(distTmp)
        angSepMask = angSepMask[0:100]'''

    angSepMask = np.argsort(distTmp)
    angSepMask = angSepMask[0:NGAL]

    #select unique galaxy ids within searchRadius
    uniqueGalIDinRadius = galIDfinal[angSepMask]
    raFinalGalInRadius  = raFinalGal[angSepMask]
    decFinalGalInRadius = decFinalGal[angSepMask]
    #########################(RA, DEC) ==> (xi, eta)#################################
    if(gnomonic):
        raFinalGalInRadius, decFinalGalInRadius, status = s2t.ds2tp(raFinalGalInRadius, decFinalGalInRadius, ra0, dec0) 
	#print "Number points in bad transformation:", np.sum(status)
    #################################################################################

    try:
        flag1 = 1
        r1 = time()
        temp = pd.Index(galIDs)
        inde = temp.get_indexer_for(uniqueGalIDinRadius) # all the detections of gal
        inde = inde[inde > -1]
        inde = np.array(inde, dtype = 'i8')
        raGalInRadius  = raGal[inde]
        decGalInRadius = decGal[inde]
	#########################(RA, DEC) ==> (xi, eta)#################################
        if(gnomonic):
            print "raGalInRadius, decGalInRadius:", raGalInRadius, decGalInRadius
            raGalInRadius, decGalInRadius, status = s2t.ds2tp(raGalInRadius, decGalInRadius, ra0, dec0) 
	    #print "Number points in bad transformation:", np.sum(status)
	#################################################################################
        galIDinRadius  = galIDs[inde]
        galMjdInRadius = galMjd[inde]

    except IndexError:
        print pixelNo, inde.size
         
    currentGalID  = galIDinRadius[0]
    currentRaGal  = [raGalInRadius[0]]
    currentDecGal = [decGalInRadius[0]]
    print "galIDinRadius:", galIDinRadius
    
    galCounter = 0 
    position = 0 
    noOfDetectionsOfGal = galIDinRadius.size
    #print "noOfDetectionsOfGal",noOfDetectionsOfGal
    
    offsetRaArray  = np.zeros(noOfDetectionsOfGal)
    offsetDecArray = np.zeros(noOfDetectionsOfGal)
    
    #run over all the objects within the searchRadius  
    t =  time()
    for k in np.arange(noOfDetectionsOfGal-1)+1:
        print "on galaxy no", k 
        ID  = galIDinRadius[k]
        ra  = raGalInRadius[k]
        dec = decGalInRadius[k]
	
        if (ID == currentGalID):
                currentRaGal.append(ra)
                currentDecGal.append(dec)
                #print "same object now"
        else:
                #make them numpy arrays
		#print "currentGalID, uniqueGalIDinRadius:", currentGalID, uniqueGalIDinRadius[galCounter]
            currentRaGal  = np.array(currentRaGal)
            currentDecGal = np.array(currentDecGal)
		#print currentRaGal, raFinalGalInRadius[uniqueGalIDinRadius==currentGalID]
            offsetRa  = currentRaGal - raFinalGalInRadius[uniqueGalIDinRadius==currentGalID][0]
            offsetDec = currentDecGal - decFinalGalInRadius[uniqueGalIDinRadius==currentGalID][0]
            offsetRaArray[position:k]  = offsetRa
            offsetDecArray[position:k] = offsetDec
		#print "offsetRa, offsetDec", offsetRa, offsetDec
                #update counters and variables
        galCounter+= 1
        position   = k
        currentGalID  = ID
        currentRaGal  = [raGalInRadius[k]]
        currentDecGal = [decGalInRadius[k]]
    #print "time taken to calculate offsets of galaxies from their final ra/dec", time()-t
    print "galCounter after loop", galCounter	

	
    # select stars inside the pixel
    #resIndexInPixel =  pixAllStar == pixelNo
    #print "pixelNo:", pixelNo, pixAllStar
    #print "sum of star number in Pixel:", np.sum(resIndexInPixel)
    
    resIndexInPixel= []
    for j in pixelIndexForStar_l:
        #print(pickPixelNo)
        resIndexInPixel_ll =  j == int(pickPixelNo*10**(-9))
        resIndexInPixel.append(resIndexInPixel_ll)
    
    #uniqueStarIDinPixel = starID[resIndexInPixel]
    #match all detections of stars with those in the pixel
    #indexInPixel = np.in1d(starIDs, uniqueStarIDinPixel)
    #obtain original ra and dec values for stars within pixel
    raStarInPixel  = raStar[resIndexInPixel]
    decStarInPixel = decStar[resIndexInPixel]
    #########################(RA, DEC) ==> (xi, eta)#################################
    if(gnomonic):
	#print "raStarInPixel, decStarInPixel:", raStarInPixel, decStarInPixel
        raStarInPixel, decStarInPixel, status = s2t.ds2tp(raStarInPixel, decStarInPixel, ra0, dec0)# transform the (RA, DEC)/degree into (xi, eta)/degree
	#print "Number points in bad transformation:", np.sum(status)
    #################################################################################
    #raErrStarInPixel  = raErrStar[resIndexInPixel]
    #decErrStarInPixel = decErrStar[resIndexInPixel]
    rMagStarInPixel = rMagStar[resIndexInPixel]
    nObsStarInPixel = nObsStar[resIndexInPixel]
    #raStarErrInPixel  = np.zeros(np.sum(resIndexInPixel))
    #decStarErrInPixel = np.zeros(np.sum(resIndexInPixel))
    starIDInPixel  = starIDs[resIndexInPixel]
    mjdStarInPixel = starMjds[resIndexInPixel]
    print 'sum number of resIndexInPixel', np.sum(resIndexInPixel)

    medianRaOffsetEpochWise = np.zeros(len(mjdBreakAt)+1)
    medianDecOffsetEpochWise = np.zeros(len(mjdBreakAt)+1)
    #avgMjd = np.zeros(len(mjdBreakAt)+1)

    if (len(mjdBreakAt)>= 1.0):
        for mjdIdx in range(0, len(mjdBreakAt)+1):
            if(mjdIdx==0):
                var = mjdBreakAt[mjdIdx]
                mjdIndexInPixel = mjdStarInPixel < var
                mjdIndex = galMjdInRadius < var
            elif(mjdIdx==len(mjdBreakAt)):
                var = mjdBreakAt[mjdIdx-1]
                mjdIndexInPixel = mjdStarInPixel >= var
                mjdIndex = galMjdInRadius >= var
            else:
                var = mjdBreakAt[mjdIdx]
                mjdIndex = (galMjdInRadius >= mjdBreakAt[mjdIdx-1]) & (galMjdInRadius < var)
                mjdIndexInPixel = (mjdStarInPixel >= mjdBreakAt[mjdIdx-1]) & (mjdStarInPixel < var)
            if (any(mjdIndex)):
                offsetRaValues  = offsetRaArray[mjdIndex]
                offsetDecValues = offsetDecArray[mjdIndex]
                medianRaOffsetEpochWise[mjdIdx]  = np.median(offsetRaValues)
                medianDecOffsetEpochWise[mjdIdx] = np.median(offsetDecValues)
                #update the ra/dec - replace old values
                raStarInPixel[mjdIndexInPixel] -= medianRaOffsetEpochWise[mjdIdx]
                decStarInPixel[mjdIndexInPixel]-= medianDecOffsetEpochWise[mjdIdx]
		#print "np.median(offsetRaValues):", np.median(offsetRaValues)
        if((offsetRaValues.size>1)):
            raErrTmp = np.sqrt((np.pi/2)/(offsetRaValues.size-1)) * \
              (0.741*(np.percentile(offsetRaValues, 75) - np.percentile(offsetRaValues, 25)))
            decErrTmp = np.sqrt((np.pi/2)/(offsetDecValues.size-1)) * \
               (0.741*(np.percentile(offsetDecValues, 75) - np.percentile(offsetDecValues, 25)))
		    #raErrStarInPixel[mjdIndexInPixel] = np.sqrt(raErrStarInPixel[mjdIndexInPixel]**2 + raErrTmp**2) # 
		    #decErrStarInPixel[mjdIndexInPixel] = np.sqrt(decErrStarInPixel[mjdIndexInPixel]**2 + decErrTmp**2) # 
                #avgMjd[var1] = (mjdStarInPixel[mjdIndexInPixel].max() - mjdStarInPixel[mjdIndexInPixel].min() )/2

    finalObjIDstar    = starIDInPixel
    finalRaArrayStar  = raStarInPixel
    finalDecArrayStar = decStarInPixel

    #########################(xi, eta) ==> (RA, DEC)#################################
    if(gnomonic): # transform the (xi, eta)/radians into (RA, DEC)/degree
        finalRaArrayStar, finalDecArrayStar = s2t.dtp2s(np.radians(finalRaArrayStar), np.radians(finalDecArrayStar), ra0, dec0)
    #################################################################################
    if(len(finalObjIDstar)<1):
        print pixelNo, inde.size
    else:
        print finalObjIDstar, finalRaArrayStar, finalDecArrayStar, mjdStarInPixel,rMagStarInPixel, nObsStarInPixel

raGalInRadius, decGalInRadius: [11.215575 11.215572 11.217781 ... 11.189387 11.23784  11.237825] [41.620327 41.620323 41.619585 ... 41.602893 41.611454 41.611463]
galIDinRadius: [770 770 725 ... 142 441 441]
on galaxy no 1
on galaxy no 2
on galaxy no 3
on galaxy no 4
on galaxy no 5
on galaxy no 6
on galaxy no 7
on galaxy no 8
on galaxy no 9
on galaxy no 10
on galaxy no 11
on galaxy no 12
on galaxy no 13
on galaxy no 14
on galaxy no 15
on galaxy no 16
on galaxy no 17
on galaxy no 18
on galaxy no 19
on galaxy no 20
on galaxy no 21
on galaxy no 22
on galaxy no 23
on galaxy no 24
on galaxy no 25
on galaxy no 26
on galaxy no 27
on galaxy no 28
on galaxy no 29
on galaxy no 30
on galaxy no 31
on galaxy no 32
on galaxy no 33
on galaxy no 34
on galaxy no 35
on galaxy no 36
on galaxy no 37
on galaxy no 38
on galaxy no 39
on galaxy no 40
on galaxy no 41
on galaxy no 42
on galaxy no 43
on galaxy no 44
on galaxy no 45
on galaxy no 46
on galaxy no 47
on galaxy no 48
on galaxy no 49
on galaxy no 50
o

on galaxy no 812
on galaxy no 813
on galaxy no 814
on galaxy no 815
on galaxy no 816
on galaxy no 817
on galaxy no 818
on galaxy no 819
on galaxy no 820
on galaxy no 821
on galaxy no 822
on galaxy no 823
on galaxy no 824
on galaxy no 825
on galaxy no 826
on galaxy no 827
on galaxy no 828
on galaxy no 829
on galaxy no 830
on galaxy no 831
on galaxy no 832
on galaxy no 833
on galaxy no 834
on galaxy no 835
on galaxy no 836
on galaxy no 837
on galaxy no 838
on galaxy no 839
on galaxy no 840
on galaxy no 841
on galaxy no 842
on galaxy no 843
on galaxy no 844
on galaxy no 845
on galaxy no 846
on galaxy no 847
on galaxy no 848
on galaxy no 849
on galaxy no 850
on galaxy no 851
on galaxy no 852
on galaxy no 853
on galaxy no 854
on galaxy no 855
on galaxy no 856
on galaxy no 857
on galaxy no 858
on galaxy no 859
on galaxy no 860
on galaxy no 861
on galaxy no 862
on galaxy no 863
on galaxy no 864
on galaxy no 865
on galaxy no 866
on galaxy no 867
on galaxy no 868
on galaxy no 869
on galaxy no 8

on galaxy no 111
on galaxy no 112
on galaxy no 113
on galaxy no 114
on galaxy no 115
on galaxy no 116
on galaxy no 117
on galaxy no 118
on galaxy no 119
on galaxy no 120
on galaxy no 121
on galaxy no 122
on galaxy no 123
on galaxy no 124
on galaxy no 125
on galaxy no 126
on galaxy no 127
on galaxy no 128
on galaxy no 129
on galaxy no 130
on galaxy no 131
on galaxy no 132
on galaxy no 133
on galaxy no 134
on galaxy no 135
on galaxy no 136
on galaxy no 137
on galaxy no 138
on galaxy no 139
on galaxy no 140
on galaxy no 141
on galaxy no 142
on galaxy no 143
on galaxy no 144
on galaxy no 145
on galaxy no 146
on galaxy no 147
on galaxy no 148
on galaxy no 149
on galaxy no 150
on galaxy no 151
on galaxy no 152
on galaxy no 153
on galaxy no 154
on galaxy no 155
on galaxy no 156
on galaxy no 157
on galaxy no 158
on galaxy no 159
on galaxy no 160
on galaxy no 161
on galaxy no 162
on galaxy no 163
on galaxy no 164
on galaxy no 165
on galaxy no 166
on galaxy no 167
on galaxy no 168
on galaxy no 1

on galaxy no 837
on galaxy no 838
on galaxy no 839
on galaxy no 840
on galaxy no 841
on galaxy no 842
on galaxy no 843
on galaxy no 844
on galaxy no 845
on galaxy no 846
on galaxy no 847
on galaxy no 848
on galaxy no 849
on galaxy no 850
on galaxy no 851
on galaxy no 852
on galaxy no 853
on galaxy no 854
on galaxy no 855
on galaxy no 856
on galaxy no 857
on galaxy no 858
on galaxy no 859
on galaxy no 860
on galaxy no 861
on galaxy no 862
on galaxy no 863
on galaxy no 864
on galaxy no 865
on galaxy no 866
on galaxy no 867
on galaxy no 868
on galaxy no 869
on galaxy no 870
on galaxy no 871
on galaxy no 872
on galaxy no 873
on galaxy no 874
on galaxy no 875
on galaxy no 876
on galaxy no 877
on galaxy no 878
on galaxy no 879
on galaxy no 880
on galaxy no 881
on galaxy no 882
on galaxy no 883
on galaxy no 884
on galaxy no 885
on galaxy no 886
on galaxy no 887
on galaxy no 888
on galaxy no 889
on galaxy no 890
on galaxy no 891
on galaxy no 892
on galaxy no 893
on galaxy no 894
on galaxy no 8

on galaxy no 777
on galaxy no 778
on galaxy no 779
on galaxy no 780
on galaxy no 781
on galaxy no 782
on galaxy no 783
on galaxy no 784
on galaxy no 785
on galaxy no 786
on galaxy no 787
on galaxy no 788
on galaxy no 789
on galaxy no 790
on galaxy no 791
on galaxy no 792
on galaxy no 793
on galaxy no 794
on galaxy no 795
on galaxy no 796
on galaxy no 797
on galaxy no 798
on galaxy no 799
on galaxy no 800
on galaxy no 801
on galaxy no 802
on galaxy no 803
on galaxy no 804
on galaxy no 805
on galaxy no 806
on galaxy no 807
on galaxy no 808
on galaxy no 809
on galaxy no 810
on galaxy no 811
on galaxy no 812
on galaxy no 813
on galaxy no 814
on galaxy no 815
on galaxy no 816
on galaxy no 817
on galaxy no 818
on galaxy no 819
on galaxy no 820
on galaxy no 821
on galaxy no 822
on galaxy no 823
on galaxy no 824
on galaxy no 825
on galaxy no 826
on galaxy no 827
on galaxy no 828
on galaxy no 829
on galaxy no 830
on galaxy no 831
on galaxy no 832
on galaxy no 833
on galaxy no 834
on galaxy no 8

on galaxy no 434
on galaxy no 435
on galaxy no 436
on galaxy no 437
on galaxy no 438
on galaxy no 439
on galaxy no 440
on galaxy no 441
on galaxy no 442
on galaxy no 443
on galaxy no 444
on galaxy no 445
on galaxy no 446
on galaxy no 447
on galaxy no 448
on galaxy no 449
on galaxy no 450
on galaxy no 451
on galaxy no 452
on galaxy no 453
on galaxy no 454
on galaxy no 455
on galaxy no 456
on galaxy no 457
on galaxy no 458
on galaxy no 459
on galaxy no 460
on galaxy no 461
on galaxy no 462
on galaxy no 463
on galaxy no 464
on galaxy no 465
on galaxy no 466
on galaxy no 467
on galaxy no 468
on galaxy no 469
on galaxy no 470
on galaxy no 471
on galaxy no 472
on galaxy no 473
on galaxy no 474
on galaxy no 475
on galaxy no 476
on galaxy no 477
on galaxy no 478
on galaxy no 479
on galaxy no 480
on galaxy no 481
on galaxy no 482
on galaxy no 483
on galaxy no 484
on galaxy no 485
on galaxy no 486
on galaxy no 487
on galaxy no 488
on galaxy no 489
on galaxy no 490
on galaxy no 491
on galaxy no 4

on galaxy no 1071
on galaxy no 1072
on galaxy no 1073
on galaxy no 1074
on galaxy no 1075
on galaxy no 1076
on galaxy no 1077
on galaxy no 1078
on galaxy no 1079
on galaxy no 1080
on galaxy no 1081
on galaxy no 1082
on galaxy no 1083
on galaxy no 1084
on galaxy no 1085
on galaxy no 1086
on galaxy no 1087
on galaxy no 1088
on galaxy no 1089
on galaxy no 1090
on galaxy no 1091
on galaxy no 1092
on galaxy no 1093
on galaxy no 1094
on galaxy no 1095
on galaxy no 1096
on galaxy no 1097
on galaxy no 1098
on galaxy no 1099
on galaxy no 1100
on galaxy no 1101
on galaxy no 1102
on galaxy no 1103
on galaxy no 1104
on galaxy no 1105
on galaxy no 1106
on galaxy no 1107
on galaxy no 1108
on galaxy no 1109
on galaxy no 1110
on galaxy no 1111
on galaxy no 1112
on galaxy no 1113
on galaxy no 1114
on galaxy no 1115
on galaxy no 1116
on galaxy no 1117
on galaxy no 1118
on galaxy no 1119
on galaxy no 1120
on galaxy no 1121
on galaxy no 1122
on galaxy no 1123
on galaxy no 1124
on galaxy no 1125
on galaxy 

KeyboardInterrupt: 

In [12]:
pmfuns.correctedStars(packParameterList[0:10000], h5correctedStarsFile)

raGalInRadius, decGalInRadius: [11.221554 11.22156  11.222209 ... 11.196169 11.1957   11.195707] [41.629223 41.629217 41.628707 ... 41.640117 41.618938 41.61894 ]
raGalInRadius, decGalInRadius: [11.217279 11.217279 11.217088 ... 11.190236 11.195974 11.195958] [41.610771 41.610763 41.611166 ... 41.616624 41.624585 41.624592]
raGalInRadius, decGalInRadius: [11.215333 11.215332 11.215183 ... 11.185263 11.222282 11.222285] [41.659115 41.659116 41.658939 ... 41.630005 41.622697 41.622693]
galIDinRadius: [ 856  856  864 ...   18 1103 1103]
galIDinRadius: [ 573  573  566 ...  981 1174 1174]
galIDinRadius: [ 370  370  376 ... 1259  627  627]
galCounter after loop 599
galCounter after loop 599
galCounter after loop 599


IndexError: invalid index to scalar variable.

In [12]:
h5file = tables.open_file(rootdir+"%s.h5" %h5fileName)

NameError: name 'h5fileName' is not defined